In [1]:
# Import python modules
import os
import datetime
import pandas as pd
import numpy as np

In [2]:
# Requires the following databses
# phenotypes_db.txt : database of genes and known phenotypes associations
# mim2gene_mod.txt : gene mim ID
# pLI.gnomad.txt : gnomad pLI calculation per gene
localfiles = "/Users/boris/GoogleDrive/UDD/research/bioinformatics/SABIO/projects/01_DECIPHERD/00_run_pipeline/"
phenotypes_db = localfiles + "/databases/phenotypes_db.txt"
mim2gene_db = localfiles + "/databases/mim2gene_mod.txt"
pLI_db = localfiles + "/databases/pLI.gnomad.txt"

In [ ]:
# Inputs
timestamp = str(datetime.datetime.now().strftime("%Y-%m-%d_%H_%M"))
run = 'run004'
prefix = "UDD034"
inputdir = localfiles + "/" + run + "/" + prefix + "_inputfiles/"
outdir = localfiles + "/" + run + "/" + prefix + "_" + timestamp + "/"
wannovar_file = inputdir + prefix + "_annovar.txt"
intervar_file = inputdir + prefix + "_intervar.txt"
phenotips_file = inputdir + prefix + "_phenotips.tsv"
sophia_file = inputdir + prefix + "_sophia.xls"

In [ ]:
# The function takes frequency values. If frequency values exist, they are compared to the AF<=0.01, otherwise, if
# no info is available, keep the site anyway
def freq_check(values, NotANum):
    results = 0
    for freq in values:
        if freq is NotANum:
            freq2 = 0
        else:
            freq2 = freq
        try:
            f = float(freq2)
            if f <= 0.01:
                results+-1
            else:
                results+=1
        except:
            results+=0
    if results <= 0:
        return "rare|unknown"
    else:
        return "common"

# The function takes impact values. If impact values exist, at least two criteria agree with functional impact, otherwise, if
# no info is available, keep the site anyway
def impact_check(values, NotANum):
    results = 0
    for impact in values:
        if impact is NotANum:
            imp2 = "D"
        else:
            imp2 = impact
        try:
            i = str(imp2)
            if i in ["D", "P", "H", "M", "A"]:
                results+=-1
            else:
                results+=1
        except:
            results+=0
    if results <= 0:
        return "affected|unknown"
    else:
        return "unaffected"

# This function check for DP of the variant call >=50
def check_DP(depth, cutoff=50):
    #dp = depth.strip().split(":")[2]
    dp = depth
    if float(dp) >= cutoff:
        return True
    else:
        return False

# This function check for GT 
def check_GT(genotype, gt_default='1/1'):
    gt = genotype.strip().split(":")[0]
    if gt == gt_default:
        return True
    else:
        return False

# This function splits DP and GT 
def split_format(fmt, n):
    out = fmt.strip().split(":")[n]
    if n in [2,3]:
        return int(out)
    else:
        return out

# Determine zygosity
def zygosity(genotype):
    A1,A2 = genotype.split("/")
    if A1==A2:
        return "hom"
    else:
        return "het"

def compound(genes, gene_occ):
    gene_list = str(genes).split(',')
    occurence = [gene_occ[gene] for gene in gene_list]
    if len([i for i in occurence if i>1]) !=0:
        return 'true'
    else:
        return 'false'

def pheno_dic(entry):
    genes = entry[0]
    pheno = entry[1]
    for gene in genes.split(','):
        return (gene,pheno)

def get_pheno(genes,phenodb):
    phenotypes = []
    for gene in str(genes).split(','):
        try:
            phenotypes.append(phenodb[gene])
        except:
            pass
    return ''.join(list(set(phenotypes)))

def get_info(info, val):
    info_values = info.split(';')
    tmp = [i for i in info_values if i.startswith(val)]
    try:
        out = float(tmp[0].split("=")[1])
    except:
        out = ''
    return out

def check_gene(genes, db):
    genelist =  str(genes).split(',')
    match = [g for g in genelist if g in db.keys()]
    if match:
        n, feat, hpo = pheno_db[match[0]]
        return pd.Series(["yes", n, feat, hpo])
    else:
        return pd.Series(["no",0,'-','-'])

In [ ]:
% mkdir {outdir}

In [ ]:
# Read genomic data table generated by uploading HC-GATK raw VCF to http://wannovar.wglab.org/ 
data = pd.read_csv(wannovar_file, low_memory=False, sep='\t')

In [ ]:
inter = pd.read_csv(intervar_file, low_memory=False, sep='\t')
inter_min = inter[[list(inter.columns)[i] for i in [0,1,2,3,4,12,13]]]
inter_min.columns = ["Chr", "Start", "End", "Ref", "Alt" , "ClinVar", "InterVar"]
clinvar_col = list(inter_min['ClinVar'].apply(lambda x: x.split(':')[1].strip()))
intervar_col = list(inter_min['InterVar'].apply(lambda x: x.split(":")[1].split("P")[0].strip()))

inter_def = inter_min.iloc[:,0:5]
inter_def['ClinVar'] = clinvar_col
inter_def['ACMG_InterVar'] = intervar_col
del inter
del inter_min
del clinvar_col
del intervar_col

clinical = inter_def.drop_duplicates()
del inter_def

In [ ]:
tmp = data.merge(clinical, how='left', on=list(clinical.columns)[:-2])
del data
del clinical
data = tmp

In [ ]:
# Add MIM phenotypes info
pheno_0 = pd.read_csv(phenotypes_db, header=None, sep='\t')
pheno_0.columns = ['Gene.refgene','Phenotypes']
pheno_1 = pheno_0.dropna()
phenodbase = dict(list(pheno_1.apply(pheno_dic, axis=1)))
data["Phenotypes"] = data['Gene.refGene'].apply(get_pheno, phenodb=phenodbase)

del pheno_0
del pheno_1
del phenotypes_db

In [ ]:
#Add MIM gene info
omim = pd.read_csv(mim2gene_db, low_memory=False, header=None, sep='\t')
omim.columns = ['mim', 'Gene.refGene']
weblinks = omim.mim.apply(lambda x: '=HYPERLINK("https://www.omim.org/entry/%s", "%s")' % (str(x), str(x)))
omim['mim'] = weblinks
data_2 = pd.merge(data, omim, how = 'left')

del omim
del mim2gene_db

In [ ]:
#Add pLI gene info
pli = pd.read_csv(pLI_db, low_memory=False, sep='\t')
pli.columns = ['Gene.refGene', 'pLI']
data_3 = pd.merge(data_2, pli, how = 'left')

del pli
del pLI_db

In [ ]:
# Add cleaner columns to data table 
del data
del data_2
data = data_3

data["Genotype"] = data["Otherinfo.12"].apply(split_format, n=0)
data["Genotype.qual"] = data["Otherinfo.12"].apply(split_format, n=3)
data["Depth"] = data["Otherinfo.12"].apply(split_format, n=2)
data["Zygosity"] = data['Genotype'].apply(zygosity)
data["Allele_counts"] = data["Otherinfo.12"].apply(split_format, n=1)
data["Site.qual"] = data["Otherinfo.8"]
data["strand.FS"] = data['Otherinfo.10'].apply(get_info, val='FS')
data["strand.OR"] = data['Otherinfo.10'].apply(get_info, val='SOR')

data["CHROM"] = data["Otherinfo.3"]
data["POS"] = data["Otherinfo.4"]
data["REF"] = data["Otherinfo.6"]
data["ALT"] = data["Otherinfo.7"]

data['Effect'] = data['GeneDetail.refGene'].str.cat(data['AAChange.refGene'], sep =",") 

In [ ]:
Snp = []
for snp, chrom, pos, ref,alt in data[['avsnp147', 'CHROM','POS','REF','ALT']].values:
    if snp!='.':
        Snp.append('=HYPERLINK("https://www.ncbi.nlm.nih.gov/snp/%s", "%s")' % (snp, snp))
    else:
        loc = "%s:%s%s>%s" % (chrom,pos,ref,alt)
        Snp.append('=HYPERLINK("https://gnomad.broadinstitute.org/region/%s-%s-%s", "%s")' % (chrom,pos,pos,loc))

data['dbSNP147'] = Snp       

In [ ]:
# transform dbSNP to weblink
#snp = data['avsnp147'].apply(lambda x: '=HYPERLINK("https://www.ncbi.nlm.nih.gov/snp/%s", "%s")' % (str(x), str(x)))
#data['dbSNP147'] = snp

In [ ]:
# Extract frequency databases column names
freq_cols_tmp = [col for col in list(data.columns) if col.endswith("_ALL") or col.endswith("_all")]
freq_cols = [freq_cols_tmp[i] for i in [0,2,3,4]]

# Extract Clinvar column
#clinVar_cols = [col for col in list(data.columns) if col.startswith("ClinVar")][1:]
clinVar_cols = ["CLNSIG"]

# Define impact criterias to be considered
impact_cols = ["SIFT_pred", "Polyphen2_HDIV_pred", "Polyphen2_HVAR_pred", 
               "MutationTaster_pred", "MutationAssessor_pred"]

In [ ]:
filter_a = ~( data["ExonicFunc.refGene"].isin(["synonymous SNV"]) 
                | data["Func.refGene"].isin(["intergenic"])
                | data["Func.refGene"].str.contains("intronic")
                | data["Func.refGene"].str.contains("stream"))

filter_b = data["Site.qual"]>=30
filter_c = data["Depth"]>=10
filter_d = data["strand.FS"]<=200
filter_e = data["strand.OR"]<=10

#reject FS>60, SOR >3 for SNPs and FS>200, SOR>10 for indels.

filter_1 = filter_a & filter_b & filter_c & filter_d & filter_d
del filter_a
del filter_b
del filter_c
del filter_d
del filter_e

In [ ]:
allele_freq = data[freq_cols].apply(freq_check, axis=1, NotANum=np.nan)

In [ ]:
pred_effect = data[impact_cols].apply(impact_check, axis=1, NotANum=np.nan)

In [ ]:
data['db_AF'] = allele_freq
data['pred_effect'] = pred_effect

In [ ]:
# Was gene suggested by Phenotips?
phenotips = pd.read_csv(phenotips_file, low_memory=False, sep='\t')
phenotips.columns = ['Gene.refGene', 'GeneID', 'Phenotips_Score', 'Features','HPOs']

DB = [tuple(x) for x in phenotips[['Gene.refGene','Phenotips_Score', 'Features','HPOs']].values]
pheno_db = dict([(x[0], x[1:4]) for x in DB])

phenotype = data['Gene.refGene'].apply(check_gene, db=pheno_db)
phenotype.columns = ['in_phenotips', 'n_phenotips', 'feat_phenotips', 'HPO_terms']

In [ ]:
data2 = pd.concat([data,phenotype], axis=1)

In [ ]:
sophia = pd.read_excel(open(sophia_file))
sophia_cols = ['chromosome','genome_position','ref','alt','Category','ACMG value']
sophia_labs = ['CHROM','POS','REF','ALT','Category_sophia','ACMG_sophia']
sophia_short = sophia[sophia_cols]
sophia_short.columns = sophia_labs
del sophia
sophia = sophia_short

In [ ]:
data3 = pd.merge(data2.astype(str), sophia.astype(str), how='left').fillna('-')

In [ ]:
out_cols = ["dbSNP147","Zygosity", "Allele_counts", "db_AF", "pred_effect",
            "Func.refGene", "ExonicFunc.refGene", "Gene.refGene", 
            "Effect", "Phenotypes", "mim", "pLI", "CADD_phred","ClinVar", "ACMG_InterVar",
            "Category_sophia","ACMG_sophia"] + list(phenotype.columns)

In [ ]:
data4 = data3[filter_1][out_cols]
data4.drop_duplicates(inplace=True)

In [ ]:
t = {'Pathogenic':1000, 'Likely pathogenic':500, 'Uncertain significance':100, 'A':1000, 'B':500, 'C':100, '3':100, '4':500, '5':1000}
score = []
for array in data4[['ACMG_InterVar', 'ACMG_sophia', 'Category_sophia', 'n_phenotips']].values:
    total = 0
    if int(array[3]) > 0:
        n = int(array[3])
    else:
        n = 1     
    for v in array[:3]:
        try:
            total += t[v]
        except:
            total += 0
    score.append(total*n)

In [ ]:
data4['score'] = score
data4.sort_values(by=['score','n_phenotips'], ascending=False, inplace=True)

In [ ]:
writer = pd.ExcelWriter(os.path.expanduser(outdir + prefix + "_" + timestamp + ".xlsx"))
data4.to_excel(writer, prefix, index=False)
writer.save()

In [ ]:
acmg = {'Benign': 1, 'Likely benign': 2, 'Uncertain significance': 3, 'Likely pathogenic' : 4, 'Pathogenic' : 5}

In [ ]:
compare_acmg = data3[(data3['ACMG_sophia'] !='-') & (data3['ACMG_InterVar'] != '')][['ACMG_InterVar','ACMG_sophia']].apply(lambda x: pd.Series([ acmg[x[0]], x[1] ]),axis=1).astype(str)

In [ ]:
# % Identity between Intervar y Sophia Genetics for ACMG classification
sum(compare_acmg[0] == compare_acmg[1])/float(len(compare_acmg))

In [ ]:
nb = os.path.expanduser(outdir + prefix + "_" + timestamp + ".ipynb")

In [ ]:
%notebook -e $nb